# Optimization automatization full - auto relaxing constrints
In this notebook you can see the optimization engine, calling a script to get the optimization (optimization engine). If the engine return an infeasible solution there is a for that relaxing the constaints until get a feasible solution (in teory)

## Root folder and read env variables

In [ ]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-1]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_GCP = os.environ.get("PROJECT_GCP", "")

## Load licence gurobi

In [ ]:
##########  LOAD LICENCE GUROBI ##########
import gurobipy as gp

# set env variable with the path of the licence
name_file_licence_gurobi = "gurobi.lic"
path_licence_gurobi = root_path + '\\' + name_file_licence_gurobi
os.environ ["GRB_LICENSE_FILE"] = path_licence_gurobi

In [ ]:
######### LAOD CONTENT LICENCE GUROBI #########
with open(path_licence_gurobi, 'r') as f:
    content_licence = f.read()
WLSACCESSID = content_licence.split('\n')[3].split('=')[1] # load WLSACCESSID (string)
WLSSECRET = content_licence.split('\n')[4].split('=')[1] # load WLSSECRET (string)
LICENSEID = int(content_licence.split('\n')[5].split("=")[1]) # load LICENSEID (integer)

params = {
"WLSACCESSID": WLSACCESSID,
"WLSSECRET": WLSSECRET,
"LICENSEID": LICENSEID
}

## RUN

In [ ]:
import pickle
import pandas as pd
import numpy as np

#gurobi
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr
import gurobipy as gp

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## IMPORTANTE LOS ARCHIVOS DE CONFIGURACIÓN SE LEEN AFUERA DEL SCRIPT DE OPTIMIZACIÓN PORQUE AFUERA ESTÁ EL CÓDIGO PARA RELAX LAS RESTRICCIONES CAMBIANDO LOS PARÁMETROS Y LUEGO EL SCRIPT RECIBE LOS VALORES EXACTOS CON LOS QUE TIENE QUE RESOLVER EL PROBLEMA DE OPTIMIZACIÓN

- **PARA SIMPLIFICAR TODO, LEO TODOS LOS ARCHIVOS DE CONFIGURACIÓN AFUERA, AUNQUE ALGUNOS NO SE RELAJEN NUNCA, COMO POR EJEMPLO LOS ARCHIVOS QUE CONFIGURAN LA RED**

### 1. Load configuration file optimizer and configuration instance to solve
The are principally two kind of files to config optimizer
- **configuration file to create optimizer**: there a files used to create the optimization network such as, list of sets, list of variables, upper bound and lower bound, etc. Pricipally in this files should be parameters that doesn't change too much across the time. For example, the list of variable, is a parameter that if change, the structure of the network change and the machine learning models needs to change too

- **configuration file with instance to solve**: there files that change its values every time that the optimizer solve the problem. It represents the files with the actual values of the features, and so, this values change every time that the optimizer is executed

#### 1.1. IndexTime file

In [ ]:
#################### define set ####################

## paths and files names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_indextime = 'IndexTime.xlsx'
path_indextime = path_folder_config_optimization + file_indextime

# read file
indextime = pd.read_excel(path_indextime)

# set index
index_set_time = pd.Index(indextime['IndexTime'].values)
index_set_time

#### 1.2 Decision variables

In [ ]:
#################### define decision variables ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_allvariables = 'AllVariables.xlsx'
path_allvariables = path_folder_config_optimization + file_allvariables

# read file
config_allvariables = pd.read_excel(path_allvariables)

# table
config_allvariables

#### 1.3 Initial Values
-  This configuration files corresponde to the description **"configuration file with instance to solve"**. This file has the initial values to start the optimizer
-  **OBS: the decision variable that are targets of machine learning models its initial value it nos defined. For all the optimization process since t=0 to t=N all the values are predicted by ml model**

In [ ]:
#################### define initial values ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_initvalues = 'InitialValues.xlsx'
path_initvalues = path_folder_config_optimization + file_initvalues

# read file
config_initvalues = pd.read_excel(path_initvalues)

# table
config_initvalues

#### OBS: at this part all the parameters of decision variables and observed variables were defined. Now, it is necesary define a structure of the optimization network to have the capacity to generate whatever network of this kind of problem with the posibilty to change the number of variables, its limits, capacity of tanks and also the NUMBER of process and tanks

#### 1.4 Define models to load
In this file are defined the path to ml models used in each process. Then reading the table the optimizer can go to load the ml model

**As you can see a process can have multiple models because the process has multiple outputs. BUT each model to develop has its own and unique name**

In [ ]:
#################### define initial values ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_modelsml = 'ModelsML.xlsx'
path_modelsml = path_folder_config_optimization + file_modelsml

# read file
config_modelsml = pd.read_excel(path_modelsml)

# table
config_modelsml

#### 1.5 Map tanks
Read a configuration table that map each tank in the process. For each tank is mapped the input flows and the output flows

Important, obviosly, the input and outputs of the tank needs to be defined in the table that maps all the features in the network

In [ ]:
#################### define map input outputs each tank ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_maptanks = 'MapTanks.xlsx'
path_maptanks = path_folder_config_optimization + file_maptanks

# read file
config_maptanks = pd.read_excel(path_maptanks)

# table
config_maptanks

#### 1.6 Map process Machine learning models features and target
When each machine learning model is trained, one of the outputs of the tranining process is the artifact with the model and also a master table with the features and target of the model.

- **In this example the individual master table for each model was generated in the training process and the global table is generated manually by the author**

- **As you can see a process can have multiple models because the process has multiple outputs. BUT each model to develop has its own and unique name**

In [ ]:
#################### define map features and targer of each model for each process ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_mapprocess_mlmodels = 'MapProcessMLmodels.xlsx'
path_mapprocess_mlmodels = path_folder_config_optimization + file_mapprocess_mlmodels

# read filemapprocess_mlmodels
config_mapprocess_mlmodels = pd.read_excel(path_mapprocess_mlmodels)

# table
config_mapprocess_mlmodels

### 2. Solve optimization
Solve optimization relaxing constraints. In this example is relaxing the rate change of decision vars

**In this first example, the rate change of decision vars are all relaxed and the unique rate change is the volume of tanks**

In [ ]:
# define list of decision vars that will have change in its rate change
list_tags_to_relax_constraints = ['TL1', 'TL2', 'TL3']
list_tags_to_relax_constraints

In [ ]:
# define factor (percent = 0.1) to change the rate change of the decision vars. 
# In this example all the decision var masked will change its value in the same factor
param_factor_relax_constraints = 0.1
factor_relax_constraints = 1 + param_factor_relax_constraints
factor_relax_constraints

In [ ]:
# print initial values of constraints that can be relaxing its values
mask = config_allvariables['feature_name'].isin(list_tags_to_relax_constraints)
config_allvariables[mask]

In [ ]:
from optimization_engine import optimization_engine

In [ ]:
""" do while. do define status solver // while status solver != 2 relaxing contrainsts (rate of change decision variables) until get a solution """
# initialize status
status_solver = 0

# initialize count iterations of relaxing constraints
index_count_while = 0
max_interations_relaxing_constraints = 40

# initiliaze values of constraints to relax
to_solver_config_allvariables = config_allvariables.copy()



while (status_solver != 2) & (index_count_while <=max_interations_relaxing_constraints):
    print('\n\n\n index while: ', index_count_while)

    ###### run optimization
    model_opt, status_solver, decision_var = optimization_engine(index_set_time,
                                    config_allvariables,
                                    config_initvalues,
                                    config_modelsml,
                                    config_maptanks,
                                    config_mapprocess_mlmodels,
                                    params)

    ##### get status solver - if solver get a solution - break the while
    if status_solver == 2:
        break
    
    # ###### relaxing constraints. if the solver return a value this values was delete, else the relaxing constraints are used in the while to get a solution
    # multiply rate change by factor of selected features
    mask = config_allvariables['feature_name'].isin(list_tags_to_relax_constraints)
    config_allvariables.loc[mask, 'rate_change'] = config_allvariables[mask]['rate_change'] * factor_relax_constraints

    # increse while
    index_count_while += 1

In [ ]:
# check status solver
print('status solver: ' , status_solver)
if status_solver != 2:
    print('Infeasible solution')

In [ ]:
##### REVISAR QUÉ VALORES DE RATE CHANGE QUEDARON DE LAS VARIABLES
mask = config_allvariables['feature_name'].isin(list_tags_to_relax_constraints)
config_allvariables[mask]

### PRINT SOLUTION

In [ ]:
# get optimal values and save in a dataframe
######## create a dataframe with set as index
solution = pd.DataFrame(index = index_set_time)

######################## save optimal values - features of models (only the features) ########################
solution["var_X1"] = decision_var['X1'].gppd.X
solution["var_O1"] = decision_var['O1'].gppd.X
solution["var_O2"] = decision_var['O2'].gppd.X
solution["var_O3"] = decision_var['O3'].gppd.X
solution["var_Y1"] = decision_var['Y1'].gppd.X
solution["var_O4"] = decision_var['O4'].gppd.X
solution["var_Z1"] = decision_var['Z1'].gppd.X
solution["var_X2"] = decision_var['X2'].gppd.X
solution["var_O5"] = decision_var['O5'].gppd.X
solution["var_O6"] = decision_var['O6'].gppd.X
solution["var_Y2"] = decision_var['Y2'].gppd.X
solution["var_Y3"] = decision_var['Y3'].gppd.X
solution["var_X3"] = decision_var['X3'].gppd.X
solution["var_O7"] = decision_var['O7'].gppd.X
solution["var_TL1"] = decision_var['TL1'].gppd.X
solution["var_TL2"] = decision_var['TL2'].gppd.X
solution["var_TL3"] = decision_var['TL3'].gppd.X

######################## # get value objetive function ########################
opt_objetive_function = model_opt.ObjVal

In [ ]:
# know status optimization
# docu: https://www.gurobi.com/documentation/current/refman/optimization_status_codes.html#sec:StatusCodes
model_opt.Status

In [ ]:
# show value objetive function
opt_objetive_function

In [ ]:
# show value decision variables
solution.T

## IMPORTANTE MI SOLUCIÓN ACTUAL ES RELAJADO LOS RATE CHANGE DE LAS VARIABLES DE DECISIÓN Y CON LOS VALORES DE LOS RATE CHANGE DE TANQUES TAMBIÉN RELAJADOS